In [3]:
from bs4 import BeautifulSoup
import csv
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
import torch.nn as nn
import re
import requests

# Initialize the BERTSUM model and tokenizer
class BERTSUM(nn.Module):
    def __init__(self, model_name='bert-base-uncased'):
        super(BERTSUM, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.linear = nn.Linear(self.bert.config.hidden_size, 1)
        self.softmax = nn.Softmax(dim=1)  # Per word softmax for importance

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        word_output = outputs.last_hidden_state  # Word token representations
        scores = self.linear(word_output)
        scores = self.softmax(scores)
        return scores.squeeze(-1)

# Load pre-trained model and tokenizer
model = BERTSUM()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to extract most important sentences based on word importance scores
def summarize_text(text, model, tokenizer, num_sentences=1):
    paragraphs = text.split('\n')
    paragraphs = [p for p in paragraphs if p.strip()]  # Hapus paragraf kosong
    important_sentences = []
    
    for paragraph in paragraphs:
        sentences = re.split(r'(?<=[.!?]) +', paragraph)  # Pisah paragraf menjadi kalimat
        paragraph_summary = []

        for sentence in sentences:
            inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
            
            # Hapus token_type_ids jika ada
            if 'token_type_ids' in inputs:
                del inputs['token_type_ids']
            
            tokenized_words = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
            
            with torch.no_grad():
                scores = model(**inputs)
            
            # Cek jika scores tidak kosong
            if scores.numel() > 0:  # Memastikan tensor tidak kosong
                k = min(5, scores.size(0))  # Set k tidak lebih besar dari ukuran tensor
                top_word_indices = torch.topk(scores.squeeze(), k=k).indices
                top_words = [tokenized_words[i.item()] for i in top_word_indices if tokenized_words[i.item()] not in tokenizer.all_special_tokens]

                # Jika kalimat memiliki kata-kata penting, tambahkan ke ringkasan paragraf
                if len(top_words) > 0:
                    paragraph_summary.append(sentence)  # Menambahkan kalimat ke ringkasan paragraf

            else:
                print("Scores tensor is empty. Skipping this sentence.")

        # Kombinasikan kalimat paling penting dari setiap paragraf
        if paragraph_summary:
            important_sentences.append(paragraph_summary[0])  # Ambil kalimat paling penting dari setiap paragraf

    # Gabungkan kalimat yang dipilih menjadi ringkasan singkat
    summary = ' '.join(important_sentences)
    return summary

# Function to scrape website and summarize
def scrape_website(url, title_tag, title_class, date_tag, date_class, content_tag, content_class):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    title = soup.find(title_tag, class_=title_class).get_text(strip=True)
    date = soup.find(date_tag, class_=date_class).get_text(strip=True)
    content = soup.find(content_tag, class_=content_class).get_text(strip=True)

    # Automatically summarize the content using BERTSUM
    summary = summarize_text(content, model, tokenizer, num_sentences=1)  # Adjust to only 1 sentence per paragraph

    return {'Title': title, 'Date': date, 'Summary': summary}

# URL and HTML tag details for the website
websites = [
    {
        'url': 'https://coinvestasi.com/berita/mayoritas-bitcoin-ternyata-sudah-tidak-aktif-lebih-dari-enam-bulan',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/studi-ungkap-97-proyek-meme-coin-alami-kegagalan-pada-2024',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/dubai-akui-kripto-untuk-bayar-gaji-pegawai',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/justin-sun-luncurkan-launchpad-meme-coin-sunpump',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/proyek-ekosistem-ton-yang-layak-diperhatikan',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/meme-coin-dogs-tentukan-jadwal-airdrop-baru-catat-tanggalnya',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/lufina-permudah-akses-investasi-real-estat-melalui-nft',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/bybit-berhasil-raih-40-juta-pengguna-di-seluruh-dunia',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/floki-jalin-kemitraan-dengan-nottingham-forest',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/arbitrum-dao-setujui-proposal-staking-token-arb',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/donald-trump-raup-keuntungan-rp112-miliar-dari-penjualan-koleksi-nft',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/krisis-suplai-listrik-iran-tawarkan-bounty-bagi-pelapor-miner-kripto-ilegal',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/vitalik-buterin-donasikan-koleksi-meme-coin-bernilai-rp83-miliar-untuk-badan-amal',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/pasar-kripto-lesu-harga-bitcoin-turun-ke-us56-000',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/coingecko-bersiap-gelar-konferensi-web3-geckocon-di-thailand',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/circle-bakal-hadirkan-fitur-pembayaran-usdc-via-nfc-di-iphone',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/ojk-targetkan-transaksi-kripto-di-2028',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/metamask-luncurkan-kartu-debit-kripto-bersama-mastercard',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/avail-rilis-mainnet-berikut-cara-menggunakan-token-avail',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/as-pindahkan-bitcoin-harga-btc-gagal-reli',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/dua-bank-investasi-ini-bocorkan-alokasi-investasi-etf-bitcoin-spot',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/game-tap-to-earn-aptos-tapos-hadir-di-telegram',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/10x-research-optimis-faktor-ini-bisa-picu-reli-harga-bitcoin',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/launchpad-meme-coin-pump-fun-dikecam-komunitas-kripto-ini-alasannya',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/pencils-protocol-rilis-soul-nft-drops-ini-cara-berpartisipasi',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/unduhan-bitget-wallet-lampaui-metamask-ini-faktor-pendukungnya',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/fbi-enggan-konfirmasi-rekam-jejak-satoshi-nakamoto',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/token-celestia-naik-jelang-pembaruan-lemongrass',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/investor-kembali-akumulasi-bitcoin-tren-bullish-mulai-mencuat',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/meme-coin-trump-kompak-ambruk-gara-gara-ini',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/bank-swasta-terbesar-di-singapura-uji-coba-token-treasury',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/marathon-digital-ingin-kumpulkan-rp39-triliun-untuk-borong-bitcoin',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/kota-bitcoin-el-salvador-raup-investasi-bernilai-rp25-triliun',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/startup-kripto-raih-dana-investasi-hingga-rp428-triliun-di-q2-2024',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/dominasi-bitcoin-turun-altcoin-diprediksi-bullish',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/token-sui-meroket-70-persen-dalam-sepekan-ini-faktor-pendorongnya',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/thailand-rilis-sandbox-untuk-dukung-inovasi-di-sektor-kripto',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/agoric-orchestration-berambisi-ciptakan-web3-terintegrasi',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/etf-ether-spot-di-as-catat-arus-dana-positif',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/cpi-juli-diprediksi-turun-harga-kripto-diharapkan-positif',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/celsius-gugat-tether-untuk-pulihkan-bitcoin-bernilai-puluhan-triliun',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/binance-diblokir-pemerintah-venezuela-imbas-kisruh-pemilu',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/ojk-pertimbangkan-penyesuaian-pajak-kripto-bersama-kemenkeu',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/ceo-cryptoquant-sebut-bitcoin-telah-kembali-ke-sinyal-bullish',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/anak-donald-trump-akan-rilis-platform-kripto-untuk-saingi-bank-tradisional',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/franklin-templeton-hadirkan-dana-tokenisasi-di-arbitrum',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/yey-quest-season-1-resmi-berakhir-1-100-pengguna-ikut-berpartisipasi',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/ojk-influencer-kripto-boleh-promosi-tapi-ada-syaratnya',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/hamster-kombat-ungkap-detail-baru-untuk-airdrop-token-hmstr',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/putin-sahkan-uu-untuk-legalkan-mining-kripto-di-rusia',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/ojk-rilis-roadmap-pengembangan-sektor-keuangan-digital-dan-kripto',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/harga-bitcoin-rebound-pulih-dari-crypto-black-monday',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/harga-ton-melejit-ini-penyebabnya',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/jumlah-investor-kripto-di-indonesia-lampaui-investor-pasar-modal-ini-tanggapan-ojk',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/wallet-bitcoin-ini-mulai-pindahkan-btc-bernilai-rp222-miliar',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/marketplace-nft-immutable-x-segera-ditutup',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/metaplanet-raih-pinjaman-rp1092-miliar-untuk-investasi-bitcoin',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/brasil-setujui-etf-spot-solana-pertama-di-dunia',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/arthur-hayes-jelaskan-pengaruh-yen-terhadap-harga-bitcoin',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/imf-jajaki-risiko-terkait-bitcoin-dengan-el-salvador',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/ripple-resmi-didenda-dalam-kasus-sec-xrp-terbang',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/cumberland-suntikan-usdt-ke-berbagai-exchange-kripto',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/volatilitas-bitcoin-naik-ke-level-tertinggi-sejak-keruntuhan-ftx',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/hacker-di-olimpiade-paris-2024-minta-tebusan-kripto',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/blackrock-dan-nasdaq-ajukan-trading-opsi-untuk-etf-ether-spot',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/trump-peringatkan-biden-untuk-tidak-jual-bitcoin-milik-as',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/diego-oliva-resign-dari-jabatan-ceo-starknet-foundation',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/bitcoin-rebound-ke-56-ribu-usd-analis-percaya-aksi-jual-mulai-berakhir',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/altcoin-potensi-cuan-agustus-2024',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/altcoin-terpuruk-gara-gara-black-monday',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/platform-defi-ini-cuan-besar-di-tengah-anjloknya-pasar-kripto',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/fbi-ungkap-penipuan-kripto-yang-mengancam-investor',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/token2049-rilis-jajaran-dj-ternama-untuk-meriahkan-acara-penutupan',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/stablecoin-tahan-uji-stres-di-tengah-gejolak-pasar-kripto',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/analis-prediksi-harga-terendah-bitcoin-dapat-sentuh-45-ribu-usd',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/total-volume-trading-etf-kripto-capai-ratusan-triliun-di-tengah-gejolak-pasar',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/galaxy-research-mayoritas-rollup-layer-2-bitcoin-tak-akan-bertahan-lama',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/investor-institusi-kompak-jual-ether-harga-eth-anjlok-ke-us2-100',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/meriahkan-olimpiade-paris-2024-plume-rilis-testnet-cultured',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/capres-as-kamala-harris-mulai-dekati-komunitas-kripto',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/pasar-kripto-ambruk-market-cap-turun-hampir-us500-miliar-dalam-tiga-hari',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/hamster-kombat-perbarui-whitepaper-bakal-distribusikan-60-token-ke-pemain',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/analis-sebut-harga-bitcoin-masih-bisa-turun-ke-50-ribu-usd',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/harga-bitcoin-ambles-ke-us54-ribu-ini-penyebabnya',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/token-ftr-dari-platform-nft-fautor-listing-di-gate-io',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/bybit-dan-ethena-luncurkan-tingkatan-imbal-hasil-agunan-usde',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/pintu-jadi-exchange-kripto-pertama-berlisensi-penuh-di-indonesia',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/3-negara-ini-percepat-program-quantum-computing-usai-terobosan-dari-china',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/harga-saham-marathon-digital-turun-pada-q2-imbas-bitcoin-halving',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/pengadilan-montenegro-tetapkan-ekstradisi-do-kwon-ke-korea-selatan',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/whale-kripto-terpantau-bersiap-sambut-reli-altcoin-ini-indikasinya',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/notcoin-luncurkan-game-telegram-baru-bersama-lost-dogs-dan-getgems',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/harga-bitcoin-alami-volatilitas-tinggi-di-tengah-kondisi-ekonomi-makro-global',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/masa-depan-dogecoin-dari-meme-coin-ke-proyek-besar',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/donald-trump-luncurkan-sneakers-bertema-bitcoin',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/platform-ini-bisa-bikin-meme-coin-dari-cuitan-di-x-twitter',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/ceo-perusahaan-kripto-dituntut-sec-terlibat-penipuan-rp41-triliun',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/volume-trading-usdc-sentuh-rp2-190-triliun-pada-juli-2024',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/grayscale-luncurkan-etf-bitcoin-spot-mini-dengan-biaya-trading-terendah',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/telegram-rilis-browser-terdesentralisasi-mini-app-store',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/the-fed-tahan-suku-bunga-as-bitcoin-turun-ke-us64-000',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/survei-ungkap-gaji-wanita-lebih-tinggi-dibanding-pria-di-industri-kripto',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/penerimaan-pajak-dari-industri-kripto-di-indonesia-hampir-sentuh-rp800-miliar',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/movement-labs-dan-polygon-labs-hadirkan-solusi-atasi-krisis-likuiditas-di-chain-l2',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/rusia-bakal-legalkan-penggunaan-kripto-dalam-perdagangan-internasional',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/polisi-ukraina-tangkap-empat-tersangka-kasus-pembunuhan-demi-bitcoin',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/dex-zkx-di-starknet-tutup-ini-penyebabnya',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/kreditor-mt-gox-pilih-hodl-daripada-jual-bitcoin',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/pendiri-tokocrypto-pang-xue-kai-bergabung-ke-drx-token',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/sec-revisi-gugatan-binance-10-token-ini-tak-lagi-dianggap-sekuritas',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/legislator-hong-kong-usulkan-bitcoin-jadi-cadangan-negara',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/pemerintah-as-pindahkan-bitcoin-sitaan-silk-road-harga-btc-merosot',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/hamster-kombat-tunda-airdrop-token-hmstr',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/harga-bitcoin-berpeluang-sentuh-us100-ribu-di-akhir-2024',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/coinshares-inflow-ke-produk-etf-eth-spot-as-capai-puluhan-triliun',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/token-unlock-terbesar-agustus-2024',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/universitas-di-as-ini-dirikan-institut-khusus-untuk-kaji-bitcoin',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/jumlah-investor-kripto-di-indonesia-tembus-20-juta-orang-per-juni-2024',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/tim-shiba-inu-bocorkan-rencana-ekosistem-dan-token-baru',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/santiment-sebut-6-altcoin-ini-berpotensi-beri-return-tinggi',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/edward-snowden-soroti-kerentanan-privasi-bitcoin',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/pidato-donald-trump-yang-pro-bitcoin-gagal-ciptakan-reli-harga',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/flipster-bagikan-tiket-coinfest-asia-2024-gratis-simak-caranya',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/5-meme-coin-bertema-kucing-yang-populer-di-2024',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/magnify-cash-rilis-token-mag-kenalkan-protokol-defi-terintegrasi-rwa',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/harga-bitcoin-rebound-jelang-pidato-trump',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/mainnet-game-web3-illuvium-rilis-token-ilv-melonjak',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/kota-ini-bakal-investasikan-dana-pensiun-ke-etf-bitcoin-spot',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/survei-kraken-sebut-bitcoin-jadi-aset-favorit-dibandingkan-meme-coin',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/etf-eth-spot-hadapi-arus-keluar-triliunan-rupiah',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/analis-ungkap-meme-coin-selebriti-di-solana-hancur-lebur',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/ton-rilis-smart-wallet-w5-perkenalkan-transaksi-bebas-gas-fee',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/mobil-sport-ferrari-bisa-dibeli-pakai-bitcoin-di-eropa',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/bitcoinos-hadirkan-zk-proof-pertama-di-mainnet-bitcoin',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/token-agoric-bld-resmi-meluncur-di-exchange-bitmart',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/perdagangan-kripto-di-indonesia-tumbuh-ratusan-persen-pada-h1-2024',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/solo-miner-raup-untung-setelah-sukses-produksi-blok-bitcoin',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/harga-bitcoin-turun-lagi-ke-us64-000-ini-pemicunya',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/kucoin-adakan-ieo-untuk-liquidswap',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/analisis-blockcircle-ungkap-waktu-tepat-untuk-beli-doge-dan-shib',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/kamala-harris-mulai-lirik-kripto-meme-coin-kama-sentuh-ath',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/bitcoin-telah-disalurkan-ke-kreditor-mt-gox',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/pajak-kripto-tinggi-india-buat-industri-lokal-meradang',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/exchange-dydx-v3-diretas-harga-token-merosot',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/volume-trading-etf-ethereum-spot-di-as-capai-triliunan-rupiah',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/yey-tech-ubah-cara-berinteraksi-komunitas-di-dunia-web3',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/bermitra-dengan-bni-dan-bri-mercuryo-rilis-fitur-beli-kripto-dengan-rupiah',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/mt-gox-pindahkan-bitcoin-harga-btc-bereaksi',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/avail-da-merilis-mainnet-dan-token-avail',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/pemerintah-as-transfer-bitcoin-yang-disita-dari-pengedar-narkoba',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/avail-umumkan-pembentukan-avail-foundation',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/zklink-resmi-luncurkan-token-zkl-harga-malah-anjlok-40',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/blackrock-catat-inflow-etf-bitcoin-rp85-triliun-dalam-sehari',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/etf-ethereum-spot-sah-diperdagangkan-di-as',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/mt-gox-aktifkan-wallet-kirim-bitcoin-ke-bitstamp',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/mog-coin-dan-popcat-pimpin-narasi-meme-coin-bertema-kucing',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/berupaya-pulihkan-kripto-yang-dicuri-wazirx-umumkan-program-bounty',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/kamala-harris-calon-kuat-pengganti-biden-bagaimana-pandangannya-soal-kripto',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/meme-coin-trump-kompak-naik-setelah-biden-mundur-dari-pilpres-as-2024',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/joe-biden-mundur-dari-pilpres-as-2024-harga-meme-coin-boden-jatuh-60',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/akun-x-pixelverse-ditangguhkan-usai-protes-penerima-airdrop-pixfi',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/binance-us-disetujui-investasikan-dana-pelanggan-ke-t-bills-as',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/presale-pepe-unchained-tembus-us43-juta-di-tengah-kenaikan-pasar-meme-coin',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/fbi-berupaya-sita-kripto-dari-penipu-di-thailand',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/minat-investor-ritel-terhadap-bitcoin-sentuh-titik-terendah-sejak-2021',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/pengguna-aktif-render-network-naik-drastis-jelang-migrasi-token',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/pixelverse-mulai-distribusi-airdrop-token-pixfi-naik',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/token-matic-polygon-segera-migrasi-ke-pol-catat-tanggalnya',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/wallet-exchange-kripto-india-wazirx-dibobol',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/the-meme-games-luncurkan-presale-token-mgmes-jelang-olimpiade-paris-2024',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/state-street-bank-tertua-kedua-di-as-ingin-rilis-stablecoin',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/volume-trading-cex-merosot-sejak-maret-2024-dex-mendominasi',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/jelang-terima-bitcoin-kreditor-mt-gox-alami-upaya-login-mencurigakan',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/vitalik-buterin-jangan-hanya-dukung-politisi-karena-pro-kripto',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/coinfest-asia-2024-segera-hadir-tawarkan-pengalaman-festival-web3-terbaik',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/bitget-token-bgb-masuk-ke-10-aset-kripto-berkinerja-terbaik-di-h1-2024',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/gara-gara-bug-pengguna-degen-chain-kehilangan-sebagian-besar-asetnya',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/bitget-token-bgb-masuk-ke-10-aset-kripto-berkinerja-terbaik-di-h1-2024',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/gara-gara-bug-pengguna-degen-chain-kehilangan-sebagian-besar-asetnya',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/ftx-dan-cftc-capai-kesepakatan',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/flipster-adakan-kompetisi-meme-bonk-ini-caranya',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/kampanye-trump-kumpulkan-donasi-kripto-hingga-rp532-miliar',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/proyek-rwa-plume-rilis-tesnet-berinsentif-ini-cara-partisipasi',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/akun-instagram-milik-exchange-kripto-asing-diblokir-kominfo',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/meme-coin-wienerai-umumkan-fase-terakhir-presale-token-wai',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/harga-bitcoin-mulai-reli-investor-fomo-jadi-pemicu',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/etf-eth-spot-diproyeksi-mulai-perdaganganpada-juli-2024',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/produk-investasi-kripto-cetak-rekor-inflow-baru',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/donald-trump-pilih-cawapres-pro-kripto',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/agoric-luncurkan-orchestration-untuk-aplikasi-web3-generasi-berikutnya',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/flipster-adakan-kompetisi-berhadiah-hingga-150-ribu-usdt',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/game-telegram-pixelverse-tetapkan-jadwal-peluncuran-token-pixfi',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/ceo-blackrock-akui-bitcoin-sebagai-instrumen-keuangan-sah',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/koin-meme-shiba-shooutout-raih-us600-ribu',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/parpol-korea-selatan-ingin-penerapan-pajak-kripto-ditunda-hingga-2028',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/12-tahun-tak-aktif-whale-bitcoin-ini-transfer-btc-senilai-rp1-triliun',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/pemerintah-jerman-resmi-jual-semua-kepemilikan-bitcoin-mereka',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/harga-bitcoin-naik-usai-insiden-dramatis-donald-trump',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/trump-gagal-dibunuh-peluang-menang-dan-memecoin-politifi-meroket',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/potensi-meme-coin-baru-dan-menarik-di-2024',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/platform-igaming-bc-game-bermitra-dengan-leicester-city',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/coinbase-sebut-pemilih-kripto-akan-berperan-besar-dalam-pemilu-as',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/biaya-transaksi-bitcoin-turun-ke-level-terendah-sejak-2020',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/analis-jpmorgan-ramal-pasar-kripto-bakal-pulih-di-agustus-2024',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/rekor-baru-eth2-staking-tembus-4736-juta-eth',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/100-situs-protokol-defi-terancam-alami-peretasan',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/argentina-lolos-final-copa-america-token-arg-naik',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/meme-coin-playdoge-berhasil-kumpulkan-us55-juta-dalam-presale',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/harga-bitcoin-stagnan-meski-inflasi-as-melambat-ini-alasannya',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/situs-compound-finance-diretas-pengguna-diimbau-waspada',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/presale-pepe-unchained-catat-pendanaan-hingga-us3-juta',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/game-cellula-rilis-program-whitelist-untuk-koleksi-nft-bitcell-inaugural',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/blackrock-akumulasi-bitcoin-di-tengah-aksi-jual-oleh-pemerintah-jerman',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/strategi-alpha-blockcircle-ungkap-potensi-proyek-kripto-octopus-network',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/goldman-sachs-akan-rilis-produk-tokenisasi-pada-akhir-2024',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/mining-kripto-ilegal-bikin-malaysia-rugi-listrik-rp117-triliun',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/survei-mayoritas-investor-tetap-optimis-terhadap-kripto-pasca-halving',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/influencer-kripto-di-indonesia-kritisi-aturan-ojk',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/base-dawgz-luncurkan-fitur-staking-untuk-pendapatan-pasif',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/etf-solana-diproyeksi-disetujui-pada-maret-2025',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/harga-btc-melemah-indeks-fear-and-greed-turun-ke-level-terendah',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/bank-italia-akan-rilis-pedoman-aset-kripto-sesuai-mica',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/ton-akan-kembangkan-layer-2-yang-kompatibel-dengan-ethereum',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/studi-ungkap-investor-kripto-cenderung-punya-kepribadian-negatif',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/mayoritas-holder-bitcoin-masih-untung-meskipun-harga-btc-anjlok',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/bitstamp-siap-prioritaskan-distribusi-bitcoin-mt-gox-ke-kreditur',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/remitansi-kripto-di-venezuela-sentuh-rp74-triliun-di-tengah-krisis-ekonomi',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/persetujuan-etf-ether-spot-tunjukan-sinyal-positif-ini-tandanya',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/sisa-bitcoin-di-wallet-pemerintah-jerman',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/buidl-blackrock-cetak-rekor-marketcap',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/lionel-messi-promosikan-memecoin-solana-water-di-instagram',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/ojk-larang-influencer-promosikan-aset-kripto',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/3-peristiwa-ekonomi-makro-penting-untuk-kripto-di-pekan-ini',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/penjualan-bitcoin-pemerintah-bukan-sebab-utama-koreksi-kripto',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/wienerai-raih-dana-us7-juta-dalam-presale-inilah-alasannya',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/metaplanet-beli-bitcoin-senilai-us24-juta-saat-ekonomi-jepang-lesu',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/tron-bakal-rilis-fitur-transfer-stablecoin-bebas-gas-fee',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/analis-ini-sebut-cardano-dan-polkadot-tak-lagi-menarik-bagi-investor-institusi',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/jerman-konsisten-jual-bitcoin',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/ceo-telegram-soroti-popularitas-hamster-kombat',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/newwit-kerja-sama-dengan-cronos-gelar-campaign-spesial-euro-2024',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/agoric-manfaatkan-cosmos-ibc-untuk-tingkatkan-transfer-lintas-rantai',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/korea-selatan-bakal-terapkan-pengawasan-transaksi-kripto-non-stop',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/jerman-didesak-berhenti-jual-bitcoin',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/webtoon-populer-solo-leveling-miliki-platform-nft',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/justin-sun-siap-beli-bitcoin-milik-jerman-demi-stabilkan-pasar',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/proses-ganti-rugi-mt-gox-pindahkan-bitcoin-senilai-us271-miliar',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/harga-bitcoin-ambruk-ke-level-us53-ribu-rp11-triliun-terlikuidasi-di-kripto',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/tanda-ini-indikasikan-harga-bitcoin-dekati-level-rendah',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/harga-btc-berpotensi-ke-us50-ribu-gagal-pertahankan-level-psikologis',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/notcoin-luncurkan-akselerator-untuk-dukung-gamefi-di-telegram',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/meme-coin-solana-catat-rata-rata-keuntungan-8-000-lampaui-ethereum',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/meme-coin-solana-mew-debut-di-game-telegram-pixeltap',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/meme-coin-pepe-unchained-peroleh-dana-us22-juta-dari-presale',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/harga-bitcoin-anjlok-ke-level-us57-ribu-terendah-sejak-mei-2024',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/rusia-berencana-legalkan-stablecoin-untuk-transaksi-lintas-negara',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/pengguna-blur-jadi-korban-phishing-nft-senilai-rp39-miliar-lenyap',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/harga-bitcoin-merosot-ke-us60-ribu-ini-pemicunya',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/jaringan-bittensor-terhenti-harga-tao-anjlok',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/narasi-ini-diproyeksi-ramaikan-kripto-di-paruh-kedua-2024',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/consensys-digugat-sec-terkait-layanan-metamask',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/analis-optimis-performa-ethereum-akan-lampaui-bitcoin-ini-alasannya',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/merger-aliansi-token-ai-resmi-masuki-fase-pertama',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/iuran-jaminan-sosial-di-filipina-bisa-dibayar-pakai-kripto',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/aset-kripto-layak-diperhatikan-juli-2024',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/as-dan-jerman-lanjut-jual-kripto-senilai-rp24-triliun-harga-btc-koreksi',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/peretas-pusat-data-nasional-ri-minta-donasi-dalam-bentuk-monero-xrm',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/trader-meme-coin-baked-sukses-raih-profit-rp49-miliar-dalam-30-menit',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/circle-jadi-penerbit-stablecoin-pertama-berlisensi-mica',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/e-commerce-jepang-adakan-acara-berhadiah-bitcoin-gratis',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/flipster-berikan-insentif-ratusan-usdt-kepada-trader-futures',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/meme-coin-base-dawgz-luncurkan-program-berhadiah-khusus-untuk-komunitas',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/kerugian-kripto-akibat-deepfake-ditaksir-hampir-us25-miliar-di-2024',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/sony-group-akuisisi-amber-group-eksplorasi-bisnis-trading-crypto',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/perusahaan-jepang-ini-borong-bitcoin-bernilai-hampir-rp20-miliar',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/3-tips-investasi-kripto-untuk-pemula-dari-ceo-binance',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/harga-bitcoin-miliki-potensi-bullish-di-juli-2024',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/arkham-ungkap-kekayaan-vitalik-buterin-segini-totalnya',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/game-telegram-pixelverse-akan-luncurkan-token-pixfi-di-ton',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/awas-ada-token-floki-inu-palsu-di-solana-dan-base',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/nilai-transaksi-kripto-di-indonesia-tembus-rp2609-triliun-sepanjang-2024',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/tindak-penghindar-pajak-kripto-malaysia-gelar-operasi-khusus',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/laporan-triple-a-38-juta-orang-di-indonesia-punya-kripto',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/kerugian-akibat-hack-dan-scam-kripto-sentuh-rp93-triliun-di-q2-2024',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/5-platform-defi-yang-layak-diperhatikan-di-2024',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/aset-kripto-dengan-performa-terbaik-di-paruh-pertama-2024',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/meme-coin-bertema-politik-terjun-bebas-pasca-debat-capres-as',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/anggota-kongres-as-usulkan-ruu-bayar-pajak-dengan-bitcoin',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/bank-bolivia-sekarang-bisa-proses-transaksi-pakai-kripto',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/vaneck-ajukan-etf-solana-spot-di-as-harga-sol-meroket',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/5-narasi-kripto-potensial-di-q3-2024',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/desa-di-jepang-luncurkan-nft-untuk-bantu-lansia',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/game-mobile-fatboy-bersiap-luncurkan-presale-token-fatty',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/token-blast-resmi-lakukan-airdrop-harga-token-langsung-reli',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/ftx-ingin-bayar-ganti-rugi-uang-tunai-kreditur-protes',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/as-janjikan-imbalan-untuk-informasi-tentang-ratu-kripto-berstatus-buron',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/pemerintah-as-terindikasi-akan-jual-bitcoin-senilai-rp4-triliun',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/token-unlock-terbesar-juli-2024',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/survei-ungkap-mayoritas-investor-institusi-jepang-mau-investasi-kripto',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/bnb-chain-sediakan-likuiditas-bernilai-rp14-miliar-untuk-kembangkan-ekosistem-meme-coin',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/harga-bitcoin-diprediksi-ke-50-ribu-usd-sebelum-reli-ini-peluang-investasinya',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/solana-hadirkan-fitur-baru-untuk-permudah-transaksi-kripto-di-media-sosial',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/akun-x-band-metallica-dibobol-dipakai-promosi-token-kripto-palsu',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/5-sektor-kripto-paling-menguntungkan-di-paruh-pertama-2024',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/bybit-geser-coinbase-sebagai-exchange-kripto-terbesar-kedua-di-dunia',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/analis-prediksi-dampak-ganti-rugi-bitcoin-mt-gox-tak-separah-perkiraan',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/solana-diterpa-rumor-sedang-diproses-penyelidikan',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/ekosistem-ton-marak-jadi-sasaran-serangan-phishing',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/pemerintah-iran-sebut-game-hamster-kombat-ancam-stabilitas-negara',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/think-thank-korea-selatan-sebut-etf-kripto-bawa-risiko',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/presiden-argentina-javier-milei-nyatakan-dukungan-terhadap-bitcoin',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
    {
        'url': 'https://coinvestasi.com/berita/rp86-miliar-dicuri-dari-exchange-kripto-turki-binance-turun-tangan',
        'title_tag': 'h1',
        'title_class': 'mb-2 h1 text-[32px] leading-[46px] lg:text-[40px] lg:leading-[54px]',        
        'date_tag': 'span',
        'date_class': 'text-gray-400 text-sm',
        'content_tag': 'div',
        'content_class': 'card-text pb-5 prose md:prose-xl detailArticle Article_detailArticle__zrRnw Article_detailHyperlink__Qp6Do'
    },
]

all_articles = []

# Collect and summarize data from each website
for website in websites:
    article = scrape_website(
        website['url'], 
        website['title_tag'], 
        website['title_class'], 
        website['date_tag'], 
        website['date_class'], 
        website['content_tag'], 
        website['content_class']
    )
    all_articles.append(article)

# Save summarized data to CSV file
csv_file = './preprocessed/coinvestasi.csv'
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['Title', 'Date', 'Summary'])
    writer.writeheader()
    for article in all_articles:
        writer.writerow(article)

# Print the contents of the CSV file
print(f"Data has been saved to {csv_file}")

df = pd.read_csv(csv_file)
print(df)

Data has been saved to ./preprocessed/coinvestasi.csv
                                                 Title  \
0    Mayoritas Bitcoin Ternyata Sudah Tidak Aktif L...   
1    Studi Ungkap 97% Proyek Meme Coin Alami Kegaga...   
2           Dubai Akui Kripto untuk Bayar Gaji Pegawai   
3     Justin Sun Luncurkan Launchpad Meme Coin SunPump   
4    5 Proyek Ekosistem TON yang Layak Diperhatikan...   
..                                                 ...   
307  Ekosistem TON Marak Jadi Sasaran Serangan Phis...   
308  Pemerintah Iran Sebut Game Hamster Kombat Anca...   
309  Think Thank Korea Selatan Sebut ETF Kripto Baw...   
310  Presiden Argentina Javier Milei Nyatakan Dukun...   
311  Rp86 Miliar Dicuri dari Exchange Kripto Turki,...   

                       Date                                            Summary  
0    Senin, 19 Agustus 2024  Sumber: Unchained/GlassnodeAngka ini menunjukk...  
1    Senin, 19 Agustus 2024  Our latest study of 30,000 projects unveils th...  
2    S